# arXiv Bilingual Reader - TranslateGemma

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jimmyliao/trans-gemma/blob/main/arxiv-reader-colab.ipynb)

**專注於雙語閱讀學習的 arXiv 論文翻譯工具**

## 🎯 核心特色

- 📖 **雙語對照閱讀**：原文與翻譯並列，學習英文學術寫作
- 🧠 **智慧混合模式**：純文字頁面快速翻譯，含圖表頁面保留視覺上下文
- 📚 **分章節翻譯**：按論文結構（Abstract, Method, Experiments）組織
- 💡 **術語對照表**：自動提取專業術語，建立中英對照
- 🇹🇼 **繁體中文優化**：確保輸出台灣慣用的繁體中文（已修復官方 bug）

## 🎓 適合誰使用？

- ✅ 研究生學習閱讀英文論文
- ✅ 研究者深入研讀重要文獻
- ✅ 準備論文寫作/審稿的學者
- ✅ 想學習英文學術寫作的人

## ⚡ 快速開始

1. 執行 Cell 1-2（環境設置 + 認證）
2. 在 Cell 3 輸入 arXiv 論文 ID 和設定
3. 執行 Cell 4 開始翻譯
4. 享受雙語對照閱讀！

---

## 👤 About the Author

<div style="display: flex; align-items: center; gap: 15px; margin: 20px 0;">
  <img src="https://sessionize.com/image/5591-400o400o2-BnJLo7cscQKXC1GqE7bFcw.jpg" alt="Jimmy Liao" style="width: 60px; height: 60px; border-radius: 50%; object-fit: cover;">
  <div>
    <strong style="font-size: 1.1em;">Jimmy Liao</strong><br>
    <span style="color: #666;">AI GDE/MVP, CTO of AI Startup</span>
  </div>
</div>



- 🐦 Twitter: [@jimmyliao](https://twitter.com/jimmyliao)
- 💼 LinkedIn: [jimmyliao](https://linkedin.com/in/jimmyliao)
- 📝 Blog: [memo.jimmyliao.net](https://memo.jimmyliao.net)
- 🔗 GitHub: [jimmyliao/trans-gemma](https://github.com/jimmyliao/trans-gemma)

---

---

## ⚠️ 重要：必須使用 GPU 加速

**TranslateGemma 需要 GPU 才能正常運作**，否則翻譯速度會非常慢：

| 硬體 | 每頁翻譯時間 | 7 頁總時間 |
|------|-------------|----------|
| ✅ **T4 GPU** | ~25 秒 | ~3 分鐘 |
| ❌ CPU | ~15-20 分鐘 | ~2-3 小時 |

### 📌 如何啟用 GPU？

**此 Notebook 已預設使用 T4 GPU**，但請確認：

1. 點擊右上角查看「連線至代管的執行階段」
2. 確認顯示 **「T4」** 而非「Python 3」
3. 如果不是 T4：
   - 點擊右上角 ▼ → 「變更執行階段類型」
   - 硬體加速器：選擇 **「T4 GPU」**
   - 點擊「儲存」

### 💡 確認 GPU 已啟用

執行以下指令確認：

```python
!nvidia-smi  # 應該看到 T4 GPU 資訊
```

---


## 💡 確認 GPU 已啟用

**執行以下指令確認：**

- ✅ 應該看到 `Tesla T4` GPU 資訊
- ❌ 如果看到錯誤訊息，請切換到 T4 GPU


In [ ]:
# ============================================
# GPU Availability Check
# ============================================

import subprocess
import sys

try:
    # 執行 nvidia-smi 確認 GPU
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=5)
    
    if result.returncode == 0:
        # GPU 可用
        output = result.stdout
        
        # 檢查是否是 T4 GPU
        if 'Tesla T4' in output or 'T4' in output:
            print("✅ GPU 已啟用：Tesla T4")
            print("\n📊 GPU 資訊：")
            # 只顯示關鍵行
            for line in output.split('\n'):
                if 'Tesla T4' in line or 'GPU Name' in line or '|' in line[:5]:
                    print(line)
            print("\n🚀 可以開始翻譯了！")
        else:
            print("⚠️  偵測到 GPU，但不是 T4")
            print("\n📊 當前 GPU：")
            for line in output.split('\n')[:15]:  # 顯示前 15 行
                print(line)
            print("\n💡 提示：其他 GPU 也可以使用，但 T4 是免費版最佳選擇")
    else:
        # nvidia-smi 執行失敗
        raise RuntimeError("nvidia-smi command failed")
        
except (subprocess.TimeoutExpired, FileNotFoundError, RuntimeError) as e:
    # GPU 不可用
    print("\n" + "="*80)
    print("❌ 錯誤：未偵測到 GPU！")
    print("="*80)
    print("\n⚠️  使用 CPU 翻譯會非常慢：")
    print("   • CPU 速度：15-20 分鐘/頁")
    print("   • T4 GPU 速度：25 秒/頁")
    print("   • 差距：~40 倍！")
    print("\n🔧 如何啟用 GPU：")
    print("   1. 點擊右上角 ▼ → 變更執行階段類型")
    print("   2. 硬體加速器：選擇『T4 GPU』")
    print("   3. 點擊『儲存』")
    print("   4. Runtime 會重新啟動")
    print("   5. 重新執行此 Cell 確認")
    print("\n" + "="*80)
    print("⛔ 程序已停止，請啟用 GPU 後重新執行")
    print("="*80 + "\n")
    
    # 拋出錯誤停止執行
    raise RuntimeError(
        "❌ GPU 未啟用！請切換到 T4 GPU 後重新執行。"
        "\n\n步驟：右上角 ▼ → 變更執行階段類型 → T4 GPU → 儲存"
    )


## 1️⃣ Setup: Clone Repository & Install Dependencies

In [ ]:
# Clone repository (single source of truth)
!rm -rf trans-gemma
!git clone https://github.com/jimmyliao/trans-gemma.git
%cd trans-gemma

# Install dependencies
!pip install uv -q
!uv pip install --system -e ".[examples]"

## 2️⃣ HuggingFace Authentication

**IMPORTANT:** TranslateGemma is a gated model. You need to:
1. Get a HuggingFace token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. Accept model access at [https://huggingface.co/google/translategemma-4b-it](https://huggingface.co/google/translategemma-4b-it)

### 🔐 Configuration Methods (Choose ONE based on your environment)

#### **Option A: Web Colab** (Using browser)
1. Click the 🔑 icon on left sidebar
2. Add secret: `HF_TOKEN` = your token
3. Run the cell below → Token loaded automatically from Colab Secrets

#### **Option B: VS Code Colab Extension** (Using VS Code locally)

**⚠️ Important:** Your local `.env` file is NOT automatically synced to remote Colab runtime!

**Solution: Create .env in remote runtime**

The cell below will:
1. First check if `.env` exists in remote runtime
2. If not found, prompt you to enter token
3. Automatically create `.env` file in remote runtime
4. Use this token for authentication

This way, you only need to enter your token once per runtime session.

#### **Option C: Manual Input Every Time** (Not recommended)
Skip .env creation and enter token manually each time.

---

**What happens when you run the cell below?**
1. Checks for `.env` in current directory (remote runtime)
2. If not found, prompts for token and creates `.env`
3. If found, reads token from `.env`
4. Authenticates with HuggingFace


In [ ]:
from huggingface_hub import login
import os
from pathlib import Path

def get_hf_token():
    """Smart HF Token retrieval with .env creation for VS Code"""
    
    # Method 1: Try .env file in current directory
    env_file = Path('.env')
    
    if env_file.exists():
        try:
            with open('.env', 'r') as f:
                for line in f:
                    line = line.strip()
                    if line.startswith('HF_TOKEN='):
                        token = line.split('=', 1)[1].strip().strip('"').strip("'")
                        if token:
                            print("✅ HF_TOKEN loaded from .env file")
                            return token
            print("⚠️  .env file found but HF_TOKEN not set correctly")
        except Exception as e:
            print(f"⚠️  Error reading .env: {e}")
    
    # Method 2: Try environment variables
    token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
    if token:
        print("✅ HF_TOKEN loaded from environment variable")
        return token
    
    # Method 3: Try Colab Secrets (Web Colab only)
    try:
        from google.colab import userdata
        token = userdata.get('HF_TOKEN')
        print("✅ HF_TOKEN loaded from Colab Secrets (Web Colab)")
        return token
    except Exception:
        pass
    
    # Method 4: Prompt for token and create .env (VS Code friendly)
    print("\n" + "="*80)
    print("⚠️  HF_TOKEN not found - Creating .env file")
    print("="*80)
    print("\n📝 Please enter your HuggingFace token:")
    print("   Get token: https://huggingface.co/settings/tokens")
    print("   Accept access: https://huggingface.co/google/translategemma-4b-it")
    print("\n💡 Your token will be saved to .env for this runtime session")
    print()
    
    token = input("HuggingFace Token: ").strip()
    
    if token:
        # Save to .env for future use in this session
        try:
            with open('.env', 'w') as f:
                f.write(f"HF_TOKEN={token}\n")
            print("\n✅ Token saved to .env file (runtime session)")
            print("   Next time you run this cell, it will load automatically")
        except Exception as e:
            print(f"\n⚠️  Could not save to .env: {e}")
            print("   Token will work this time but won't persist")
        
        return token
    else:
        raise ValueError("❌ HF_TOKEN is required to use TranslateGemma")

# Authenticate
try:
    HF_TOKEN = get_hf_token()
    os.environ['HF_TOKEN'] = HF_TOKEN
    login(token=HF_TOKEN)
    print("\n✅ Successfully authenticated with HuggingFace\n")
except Exception as e:
    print(f"\n❌ Authentication failed: {e}\n")
    raise

## 3️⃣ Configuration: Bilingual Reading Settings

**配置你的論文和翻譯設定**

In [ ]:
# ============================================
# arXiv Paper Configuration
# ============================================

# arXiv Paper ID (from URL: https://arxiv.org/abs/XXXX.XXXXX)
ARXIV_ID = "2601.09012v2"  # TranslateGemma Technical Report

# ============================================
# Bilingual Reading Settings
# ============================================

# Enable bilingual mode (always recommended for learning)
BILINGUAL_MODE = True

# Target language
TARGET_LANG = "zh-TW"  # Traditional Chinese
SOURCE_LANG = "en"     # English

# ============================================
# Translation Sections (Page Ranges)
# ============================================

# 🚀 FULL TRANSLATION MODE
# Define which sections to translate
# Format: "section_name": (start_page, end_page)
SECTIONS = {
    "abstract": (1, 1),      # Abstract (Page 1)
    "method": (3, 5),        # Method section (Pages 3-5)
    "experiments": (7, 9),   # Experiments (Pages 7-9)
}

# ============================================
# Display Settings
# ============================================

# Extract technical terms for glossary
EXTRACT_TERMS = True

# Display options
DISPLAY_MODE = "rich"  # Options: "simple", "rich", "html"
# - simple: Plain text (fast)
# - rich: Colored boxes with side-by-side layout (recommended)
# - html: HTML table (best for saving/sharing)

# Save output to file
SAVE_TO_FILE = True  # Save translation to HTML file

# Debug mode (set to True to see detailed progress)
DEBUG_MODE = False  # Enable detailed debug output

# ============================================
# Execution Mode (選擇執行模式)
# ============================================

# 選擇執行模式：
# - "arxiv": 翻譯 arXiv 論文（使用上方的 ARXIV_ID）
# - "upload": 上傳本地 PDF 檔案翻譯
# - "batch": 批次翻譯多篇論文
# - "skip": 跳過翻譯（只載入模型）

MODE = "arxiv"  # 預設：翻譯 arXiv 論文

# 批次處理設定（只在 MODE = "batch" 時使用）
BATCH_PAPERS = [
    # Format: (arxiv_id, sections_dict)
    # Example:
    # ("2601.09012v2", {"abstract": (1, 1)}),
    # ("2312.11805v1", {"abstract": (1, 1)}),
]

# Upload PDF 設定（只在 MODE = "upload" 時使用）
UPLOAD_SECTIONS = {
    "section1": (1, 3),  # 修改為你要翻譯的頁碼範圍
}
TRANSLATE_DEBUG = False  # Show full translation output (for diagnosing extraction issues)

# ============================================
# Performance Note
# ============================================
# Using text-only mode for better quality and speed
# Text mode: ~20-25s/page
# All pages will be processed as pure text

# ============================================
# Summary
# ============================================

print("\n" + "="*80)
print("🎯 arXiv Bilingual Reader - Configuration")
print("="*80)
print(f"\n📄 Paper: arXiv:{ARXIV_ID}")
print(f"🌐 Translation: {SOURCE_LANG} → {TARGET_LANG}")
print(f"📖 Bilingual Mode: {'✅ Enabled' if BILINGUAL_MODE else '❌ Disabled'}")
print(f"📺 Display Mode: {DISPLAY_MODE}")
print(f"💾 Save to file: {'✅ Yes' if SAVE_TO_FILE else '❌ No'}")
print(f"\n📚 Sections to translate:")
total_pages = 0
for section, (start, end) in SECTIONS.items():
    page_count = end - start + 1
    total_pages += page_count
    pages = f"Page {start}" if start == end else f"Pages {start}-{end}"
    print(f"   • {section.capitalize()}: {pages} ({page_count} pages)")
print(f"\n🚀 FULL MODE: Translating {total_pages} pages")
print(f"⚡ Mode: Text-only (fast and accurate)")
print(f"   Estimated time: ~{int(total_pages * 23 / 60)} minutes {(total_pages * 23) % 60} seconds")
print(f"\n💡 Ready to translate! Execute Cell 4 to start.")
print("="*80 + "\n")

## 4️⃣-1 Helper Functions

**定義輔助函數（內部使用，無需修改）**


In [ ]:
import sys
import time
import gc
import torch
from pathlib import Path

# Add examples to path
sys.path.insert(0, 'examples')
sys.path.insert(0, 'examples/backends')

# Import backends
from transformers_backend import TransformersBackend

# Import utilities
import urllib.request
import fitz  # PyMuPDF
import re
from IPython.display import HTML, display

# ============================================
# Install and import tqdm for progress bar
# ============================================
try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm -q
    from tqdm.auto import tqdm

# ============================================
# Helper Functions
# ============================================
def download_arxiv(arxiv_id):
    """Download arXiv PDF"""
    clean_id = arxiv_id.split('v')[0] if 'v' in arxiv_id else arxiv_id
    url = f"https://arxiv.org/pdf/{clean_id}.pdf"
    filename = f"{arxiv_id}.pdf"
    
    print(f"📥 Downloading from arXiv: {arxiv_id}")
    urllib.request.urlretrieve(url, filename)
    
    doc = fitz.open(filename)
    total_pages = len(doc)
    doc.close()
    
    print(f"✅ Downloaded: {filename} ({total_pages} pages)\n")
    return filename, total_pages

def upload_pdf():
    """Upload PDF file from local machine"""
    from google.colab import files
    import os
    
    print("📤 請上傳 PDF 檔案...")
    uploaded = files.upload()
    
    if not uploaded:
        raise ValueError("❌ 未選擇檔案")
    
    # 取得第一個上傳的檔案
    filename = list(uploaded.keys())[0]
    
    if not filename.endswith(".pdf"):
        raise ValueError("❌ 請上傳 PDF 檔案")
    
    doc = fitz.open(filename)
    total_pages = len(doc)
    doc.close()
    
    print(f"✅ 已上傳: {filename} ({total_pages} pages)\n")
    return filename, total_pages


## 4️⃣ Load Translation Model

**⚠️ 只需執行一次！**

- 模型載入後會保留在記憶體中
- 翻譯不同論文時**不需要重新執行此 Cell**
- 只有在 Runtime 重啟後才需要重新載入


In [ ]:
import sys
import os

# Set debug environment variable
if TRANSLATE_DEBUG:
    os.environ['TRANSLATE_DEBUG'] = '1'

# Add examples to path
sys.path.insert(0, 'examples')
sys.path.insert(0, 'examples/backends')

# Import backends
from transformers_backend import TransformersBackend

# Load text backend
print('🔄 Loading TranslateGemma model...\n')
text_backend = TransformersBackend()
text_backend.load_model()
print('✅ TranslateGemma model ready!\n')
print('💡 模型已載入，可以開始翻譯論文了！')
print('   下一步：執行 Cell 5 (Translate arXiv Paper)\n')


## 5️⃣ Translate arXiv Paper

**📝 更換論文？只需修改 ARXIV_ID 並重新執行此 Cell！**

不需要重新載入模型，直接修改下方的參數即可：

```python
# 範例：換成其他論文
translate_arxiv_paper(
    arxiv_id="2312.11805v1",  # 改這裡！
    sections={"abstract": (1, 1)},
    target_lang="zh-TW"
)
```


In [ ]:
def translate_arxiv_paper(
    arxiv_id,
    sections,
    source_lang="en",
    target_lang="zh-TW",
    bilingual_mode=True,
    extract_terms=True,
    save_to_file=True,
    display_mode="rich"
):
    """Translate an arXiv paper
    
    Args:
        arxiv_id: arXiv paper ID (e.g., '2601.09012v2')
        sections: Dict of section_name: (start_page, end_page)
        source_lang: Source language code (default: 'en')
        target_lang: Target language code (default: 'zh-TW')
        bilingual_mode: Show original and translation side-by-side
        extract_terms: Extract technical terms for glossary
        save_to_file: Save translation to HTML file
        display_mode: 'simple', 'rich', or 'html'
    
    Returns:
        results: List of translation results
    """
    import time
    import gc
    import torch
    
    print("\n" + "#"*80)
    print("# arXiv Bilingual Reader - Translation Started")
    print("#"*80 + "\n")
    
    # Download arXiv paper
    pdf_path, total_pages = download_arxiv(arxiv_id)
    
    # Calculate pages to translate
    total_pages_to_translate = sum(end - start + 1 for start, end in sections.values())
    
    print(f"📊 Translation plan:")
    print(f"   • Mode: Text-only (fast and accurate)")
    print(f"   • Total pages: {total_pages_to_translate}")
    print(f"   • Display: {display_mode}")
    print(f"   • Estimated time: ~{total_pages_to_translate * 23} seconds\n")
    
    all_terms = {}
    results = []
    
    # Translate all pages
    print("\n" + "#"*80)
    print("# Translation in Progress")
    print("#"*80 + "\n")
    
    with tqdm(total=total_pages_to_translate, desc="📖 Translating", unit="page") as pbar:
        for section_name, (start_page, end_page) in sections.items():
            for page_num in range(start_page, end_page + 1):
                pbar.set_description(f"📖 {section_name.capitalize()} - Page {page_num}/{total_pages}")
                debug_print(f"Starting page {page_num} in section {section_name}")
                
                original_text = extract_text_from_page(pdf_path, page_num)
                debug_print(f"Extracted {len(original_text)} chars from page {page_num}")
                debug_print(f"Text preview: {original_text[:150]}...")
                
                if not original_text:
                    debug_print(f"Page {page_num} has no text, skipping")
                    print(f"⚠️  Page {page_num}: No text found, skipping...\n")
                    pbar.update(1)
                    continue
                
                start_time = time.time()
                debug_print(f"Calling translation API for page {page_num}...")
                result = text_backend.translate(
                    original_text,
                    source_lang=source_lang,
                    target_lang=target_lang
                )
                time_taken = time.time() - start_time
                translation = result['translation']
                debug_print(f"Translation received: {len(translation)} chars")
                debug_print(f"Translation preview: {translation[:150]}...")
                
                # Store result
                results.append({
                    'page_num': page_num,
                    'section': section_name,
                    'original': original_text,
                    'translation': translation,
                    'time': time_taken
                })
                debug_print(f"Stored result for page {page_num}")
                
                # Extract terms
                if extract_terms:
                    terms = extract_technical_terms(original_text, translation)
                    all_terms.update(terms)
                
                pbar.update(1)
    
    # Display Results
    print("\n" + "#"*80)
    print("# Translation Results - Interactive Preview")
    print("#"*80 + "\n")
    
    if len(results) > 1:
        print(f"📄 共 {len(results)} 頁翻譯結果")
        print(f"💡 向下滾動查看所有頁面\n")
    
    # Display each page
    for idx, result in enumerate(results):
        display_bilingual_rich([result], 0)
    
    # Display technical terms glossary
    if extract_terms and all_terms:
        print("\n" + "#"*80)
        print("# 📚 Technical Terms Glossary")
        print("#"*80 + "\n")
        
        print("="*80)
        print("專業術語對照表")
        print("="*80)
        
        for en, zh in sorted(all_terms.items()):
            print(f"{en:40} → {zh}")
        
        print("\n" + "="*80)
        print(f"Total terms extracted: {len(all_terms)}")
        print("="*80)
    
    # Save Interactive HTML
    if save_to_file:
        html_output = generate_interactive_html(results, arxiv_id, source_lang, target_lang, all_terms)
        output_filename = f"translation_{arxiv_id}_{source_lang}-{target_lang}.html"
        
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(html_output)
        
        print(f"\n💾 Interactive HTML saved to: {output_filename}")
        print(f"\n✨ Features:")
        print(f"   • 左右箭頭按鈕切換頁面")
        print(f"   • 鍵盤 ← → 方向鍵快速導航")
        print(f"   • 固定導航欄顯示當前頁面")
        print(f"\n📥 使用下方的下載 cell 下載檔案\n")
    
    print("\n" + "#"*80)
    print("# ✅ Translation Complete!")
    print("#"*80 + "\n")
    print("💡 Summary:")
    print(f"   • Translated {total_pages_to_translate} pages")
    print(f"   • Sections: {', '.join(sections.keys())}")
    print(f"   • Technical terms found: {len(all_terms)}")
    print(f"   • Display mode: {display_mode}")
    if save_to_file:
        print(f"   • Interactive HTML: {output_filename}")
    
    return results

# ============================================


def translate_uploaded_pdf(
    sections,
    source_lang="en",
    target_lang="zh-TW",
    bilingual_mode=True,
    extract_terms=True,
    save_to_file=True,
    display_mode="rich"
):
    """Translate an uploaded PDF file"""
    import time
    
    print("\n" + "#"*80)
    print("# PDF Bilingual Translator - Upload Mode")
    print("#"*80 + "\n")
    
    # Upload PDF
    pdf_path, total_pages = upload_pdf()
    
    # Use same translation logic as arxiv
    # 複用 translate_arxiv_paper 的核心邏輯
    total_pages_to_translate = sum(end - start + 1 for start, end in sections.values())
    
    print(f"📊 Translation plan:")
    print(f"   • Mode: Text-only")
    print(f"   • Total pages: {total_pages_to_translate}")
    print(f"   • Estimated time: ~{total_pages_to_translate * 23} seconds\n")
    
    all_terms = {}
    results = []
    
    with tqdm(total=total_pages_to_translate, desc="📖 Translating", unit="page") as pbar:
        for section_name, (start_page, end_page) in sections.items():
            for page_num in range(start_page, end_page + 1):
                pbar.set_description(f"📖 {section_name} - Page {page_num}/{total_pages}")
                
                original_text = extract_text_from_page(pdf_path, page_num)
                if not original_text:
                    pbar.update(1)
                    continue
                
                start_time = time.time()
                result = text_backend.translate(original_text, source_lang=source_lang, target_lang=target_lang)
                time_taken = time.time() - start_time
                
                results.append({
                    "page_num": page_num,
                    "section": section_name,
                    "original": original_text,
                    "translation": result["translation"],
                    "time": time_taken
                })
                
                if extract_terms:
                    all_terms.update(extract_technical_terms(original_text, result["translation"]))
                
                pbar.update(1)
    
    # Display and save (same as arxiv version)
    for idx, result in enumerate(results):
        display_bilingual_rich([result], 0)
    
    if save_to_file:
        import os
        pdf_basename = os.path.splitext(os.path.basename(pdf_path))[0]
        html_output = generate_interactive_html(results, pdf_basename, source_lang, target_lang, all_terms)
        output_filename = f"translation_{pdf_basename}_{source_lang}-{target_lang}.html"
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_output)
        print(f"\n💾 Saved: {output_filename}")
    
    print("\n✅ Translation complete!")
    return results

def batch_translate_papers(papers_list, source_lang="en", target_lang="zh-TW"):
    """Batch translate multiple arXiv papers"""
    print(f"\n🚀 批次處理 {len(papers_list)} 篇論文...\n")
    
    all_results = []
    
    for idx, (arxiv_id, sections) in enumerate(papers_list, 1):
        print(f"\n" + "="*80)
        print(f"📄 Paper {idx}/{len(papers_list)}: {arxiv_id}")
        print("="*80 + "\n")
        
        try:
            results = translate_arxiv_paper(
                arxiv_id=arxiv_id,
                sections=sections,
                source_lang=source_lang,
                target_lang=target_lang,
                bilingual_mode=BILINGUAL_MODE,
                extract_terms=EXTRACT_TERMS,
                save_to_file=SAVE_TO_FILE,
                display_mode="simple"
            )
            all_results.append((arxiv_id, results))
            print(f"\n✅ Completed: {arxiv_id}\n")
        except Exception as e:
            print(f"\n❌ Error: {e}\n")
    
    print(f"\n🎉 批次處理完成！{len(all_results)}/{len(papers_list)} 篇成功")
    return all_results

print("✅ All translation functions loaded:")
print("   • translate_arxiv_paper()")
print("   • translate_uploaded_pdf()")
print("   • batch_translate_papers()")


## 6️⃣ Batch Processing (Optional)

**一次翻譯多篇論文**

如果你需要翻譯多篇論文，可以使用批次處理功能：

```python
papers = [
    ("2601.09012v2", {"abstract": (1, 1)}),  # TranslateGemma
    ("2312.11805v1", {"abstract": (1, 1)}),  # Gemini
    ("2204.02311v1", {"abstract": (1, 1)}),  # PaLM
]

for arxiv_id, sections in papers:
    translate_arxiv_paper(arxiv_id, sections)
```


## 5️⃣ Execute Translation

**根據 Cell 3 的 MODE 設定自動執行對應的翻譯模式**

- 🔵 `MODE = "arxiv"`: 翻譯單篇 arXiv 論文
- 🟢 `MODE = "upload"`: 上傳本地 PDF 檔案翻譯
- 🟡 `MODE = "batch"`: 批次翻譯多篇論文
- ⚪ `MODE = "skip"`: 跳過翻譯（僅載入模型）

**只需執行此 Cell，會自動根據 MODE 選擇正確的翻譯方式！**


In [ ]:
# ============================================
# Unified Translation Execution
# ============================================

print(f"\n🎯 執行模式：{MODE}\n")

if MODE == "arxiv":
    # ========================================
    # Mode 1: 翻譯 arXiv 論文
    # ========================================
    print("📄 模式：翻譯 arXiv 論文")
    print(f"   論文 ID：{ARXIV_ID}")
    print(f"   章節：{list(SECTIONS.keys())}\n")
    
    results = translate_arxiv_paper(
        arxiv_id=ARXIV_ID,
        sections=SECTIONS,
        source_lang=SOURCE_LANG,
        target_lang=TARGET_LANG,
        bilingual_mode=BILINGUAL_MODE,
        extract_terms=EXTRACT_TERMS,
        save_to_file=SAVE_TO_FILE,
        display_mode=DISPLAY_MODE
    )

elif MODE == "upload":
    # ========================================
    # Mode 2: 上傳本地 PDF
    # ========================================
    print("📤 模式：上傳本地 PDF 檔案")
    print(f"   章節：{list(UPLOAD_SECTIONS.keys())}\n")
    
    results = translate_uploaded_pdf(
        sections=UPLOAD_SECTIONS,
        source_lang=SOURCE_LANG,
        target_lang=TARGET_LANG,
        bilingual_mode=BILINGUAL_MODE,
        extract_terms=EXTRACT_TERMS,
        save_to_file=SAVE_TO_FILE,
        display_mode=DISPLAY_MODE
    )

elif MODE == "batch":
    # ========================================
    # Mode 3: 批次翻譯
    # ========================================
    if not BATCH_PAPERS:
        print("⚠️  批次處理模式：請在 Cell 3 的 BATCH_PAPERS 清單中加入論文")
        print("\n範例：")
        print('BATCH_PAPERS = [')
        print('    ("2601.09012v2", {"abstract": (1, 1)}),')
        print('    ("2312.11805v1", {"abstract": (1, 1)}),')
        print(']')
    else:
        print("📚 模式：批次翻譯多篇論文")
        print(f"   論文數量：{len(BATCH_PAPERS)}\n")
        
        results = batch_translate_papers(
            papers_list=BATCH_PAPERS,
            source_lang=SOURCE_LANG,
            target_lang=TARGET_LANG
        )

elif MODE == "skip":
    # ========================================
    # Mode 4: 跳過翻譯
    # ========================================
    print("⏭️  模式：跳過翻譯")
    print("   模型已載入，未執行翻譯")
    print("\n💡 提示：")
    print("   - 可以手動呼叫 translate_arxiv_paper()")
    print("   - 可以手動呼叫 translate_uploaded_pdf()")
    print("   - 可以手動呼叫 batch_translate_papers()")
    results = None

else:
    # ========================================
    # Invalid mode
    # ========================================
    print(f"❌ 錯誤：未知的 MODE = '{MODE}'")
    print("\n有效的選項：")
    print('  - MODE = "arxiv"   # 翻譯 arXiv 論文')
    print('  - MODE = "upload"  # 上傳本地 PDF')
    print('  - MODE = "batch"   # 批次翻譯')
    print('  - MODE = "skip"    # 跳過翻譯')
    results = None


In [ ]:
# ============================================
# Batch Processing Function
# ============================================

def batch_translate_papers(papers_list, source_lang="en", target_lang="zh-TW"):
    """批次翻譯多篇論文
    
    Args:
        papers_list: List of (arxiv_id, sections_dict) tuples
        source_lang: Source language
        target_lang: Target language
    
    Returns:
        List of (arxiv_id, results) tuples
    """
    print(f"\n🚀 開始批次處理 {len(papers_list)} 篇論文...\n")
    
    all_results = []
    
    for idx, (arxiv_id, sections) in enumerate(papers_list, 1):
        print(f"\n" + "="*80)
        print(f"📄 Paper {idx}/{len(papers_list)}: arXiv:{arxiv_id}")
        print("="*80 + "\n")
        
        try:
            results = translate_arxiv_paper(
                arxiv_id=arxiv_id,
                sections=sections,
                source_lang=source_lang,
                target_lang=target_lang,
                bilingual_mode=BILINGUAL_MODE,
                extract_terms=EXTRACT_TERMS,
                save_to_file=SAVE_TO_FILE,
                display_mode="simple"  # 批次處理用簡單模式
            )
            all_results.append((arxiv_id, results))
            print(f"\n✅ Paper {idx} complete: {arxiv_id}\n")
            
        except Exception as e:
            print(f"\n❌ Error processing {arxiv_id}: {e}\n")
            continue
    
    print("\n" + "="*80)
    print(f"🎉 批次處理完成！共處理 {len(all_results)}/{len(papers_list)} 篇論文")
    print("="*80)
    
    # 顯示所有生成的檔案
    print("\n📂 Generated Files:")
    for arxiv_id, _ in all_results:
        filename = f"translation_{arxiv_id}_{source_lang}-{target_lang}.html"
        print(f"   • {filename}")
    
    return all_results

# ============================================
# Usage Example (取消註解來使用)
# ============================================

# papers = [
#     ("2601.09012v2", {"abstract": (1, 1)}),  # TranslateGemma
#     ("2312.11805v1", {"abstract": (1, 1)}),  # Gemini
#     ("2204.02311v1", {"abstract": (1, 1)}),  # PaLM
# ]
# 
# results = batch_translate_papers(
#     papers_list=papers,
#     source_lang="en",
#     target_lang="zh-TW"
# )

print("💡 批次處理函數已載入")
print("   使用方式：取消上方註解或呼叫 batch_translate_papers()")


## 📥 Download Translation

**下載翻譯結果到本地**

Execute the cell below to download the HTML file to your local machine.

In [ ]:
from google.colab import files
import os

# Find HTML translation files
html_files = [f for f in os.listdir('.') if f.startswith('translation_') and f.endswith('.html')]

if html_files:
    print(f"📄 找到 {len(html_files)} 個翻譯檔案：\n")
    
    for html_file in html_files:
        file_size = os.path.getsize(html_file) / 1024  # KB
        print(f"   • {html_file} ({file_size:.1f} KB)")
    
    # Download the most recent one
    html_file = html_files[0]
    print(f"\n📥 下載檔案：{html_file}")
    print("   檔案會出現在你的瀏覽器下載資料夾...\n")
    
    files.download(html_file)
    
    print("\n✅ 下載完成！")
    print("\n💡 接下來：")
    print("   1. 到瀏覽器的下載資料夾找到檔案")
    print("   2. 雙擊檔案用瀏覽器開啟")
    print("   3. 享受完美排版的雙語翻譯！")
else:
    print("❌ 找不到 HTML 檔案")
    print(f"\n當前目錄：{os.getcwd()}")
    print(f"\n所有檔案：")
    for f in sorted(os.listdir('.')):
        print(f"   • {f}")